In [20]:
import re
import pandas as pd
import numpy as np
import openai
import json
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

### Calling GPT-4

In [3]:
openai.api_key = ''

In [77]:
system_prompt = '''
You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of 15 to 30 words, followed by assessment.

1. analysis_of_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren't>
2. financial_or_business_news (True or False) : <True or False based on previous attribute>
3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example international articles about 401k or small foreign companies won't be relevant for india. however changes to fed interest rates or nasdaq or important news of large multinational corporations will be relevant>
4. relevant_for_india (True or False) : <True or False based on previous attribute>
5. analysis_of_article_validity_duration (short text) : <analyse relevance duration. Be stingy: Stock fluctuations, 1 day; significant policy changes - few days or a week; educational content with references to any regulations is 30 unless there are none - in which case is timeless. International news in India has shorter lifespan. breaking news are usually not timeless; quarterly analysis or results are usually valid for a 3 days, yearly analysis or results for a weeks and a much longer one for a month.>
6. article_validity_duration (one of 1, 3, 7, 14, 30, -1) : <calculate number of days based on previous attribute. -1: timeless. 1: article is relevant only for that day. 3: for a couple of days. 7: for a week. 14: for a couple of weeks. 30: for a month>
7. analysis_of_popularity (short text) : <analyse likely popularity of article - if its for niche audience, moderate_popularity or should be part of breaking_news section, depending on number of people who will be impacted by the news and the scale of the event. foreign entities known in india but not very popular will be mostly niche or rarely moderately popular. articles targeted to very specific business or pratices will be niche. infotainment business and financial articles with some drama are likely to be more popular. articles with a list of rules without compelling story-telling will be for niche audience>
8. popularity (one of niche, moderately_popular, breaking_news) : <based on previous attribute>
9. analysis_of_article_type (short text) : <analyse if the article is majorly factual, is an opinion piece, analysis, educational or likely sponsored. factual articles pass on information on events. opinion pieces have inferences or predictions either from the author or from statements without data. analysis pieces have substantial data to justify their inferences or predictions. if an article is overly zealous on certain stock and seems like an ad, then it is sponsored>
10. article_type (one of fact, opinion, analysis, educational, sponsored) : <based on previous attribute>
11. analysis_of_article_sentiment (short_text): <analyse if the sentiment of the article is bullish, bearish or NA. balanced is NA>
12. article_sentiment (one of bull, bear, NA): <based on previous attribute>
13. headline_suggestion (short text) : <Write a headline based on the content of the article>
14. first_attempt_summary (text of 60 words) : <Generate concise, entity-dense summary. The summary should become highly dense but easily understood without the Article. Don't keep the summary too short, but limit it to no more than 60 words>
15. improved_summary (text of 60 words): <Identify contents of the article which are missing from the previous summary but are important part of the article>
16. final_summary (text of 60 words): <The finalised summary which is a mixture of first_attempt_summary and improved_summary. This summary should be very dense and also all the important information of the article and yet concise. By reading this in most cases the users need not read the article>
17. categories_hierarchy (5 semi colon seperated categories): <Hierarchy of 5 categories or keywords. Start with a generic category and make it progressively specific. Select only 5 and seperate them by (;). Don't use either single or double quotes at any cost to avoid json.loads() failure>

your response should be a json structure with all the 17 above keys without missing any key. It is very important that the response is directly readable with json.loads(). no preamble or postamble. respond in the exact following structure:

{
"analysis_of_financial_or_business_news": "",
"financial_or_business_news": "",
"analysis_of_relevant_for_india": "",
"relevant_for_india": "",
"analysis_of_article_validity_duration": "",
"article_validity_duration": "",
"analysis_of_popularity": "",
"popularity": "",
"analysis_of_article_type": "",
"article_type": "",
"analysis_of_article_sentiment": "",
"article_sentiment": "",
"headline_suggestion": "",
"first_attempt_summary": "",
"improved_summary": "",
"final_summary": "",
"top_categories": ""
}

|article_start|
'''

In [5]:
shortlisted_articles = pd.read_csv('shortlisted_articles_30_Nov_23.csv')

In [6]:
def calculate_tokens(text, encoder):  # Placeholder for your actual token calculation function
    # Your implementation will go here.
    return len(encoder.encode(text))  # Example: counting characters as tokens

def truncate_text_for_token_limit(text,encoder, token_limit):
    # First, check if the whole text is under the token limit
    if calculate_tokens(text, encoder) <= token_limit:
        return text  # The entire text is within the limit

    def is_under_limit(index):
        # Use the provided function to calculate tokens for the substring
        return calculate_tokens(text[:index], encoder) <= token_limit

    left, right = 0, len(text)
    valid_limit = 0  # This will hold the index of the last valid token position

    # Binary search to find the token limit
    while left <= right:
        mid = (left + right) // 2  # Find the midpoint
        if is_under_limit(mid):
            # If the midpoint is under the limit, store it as a valid limit
            valid_limit = mid
            left = mid + 1  # Move the left boundary to the right
        else:
            right = mid - 1  # Move the right boundary to the left

    # Find the last space before the valid_limit to ensure we're at a word boundary
    space_index = text.rfind(' ', 0, valid_limit)
    if space_index == -1:
        # If there's no space, we've hit the start of the text
        return text[:valid_limit]  # Return up to the valid limit even if mid-word

    # Return the text up to the last word within the token limit
    return text[:space_index]

In [7]:
model = 'gpt-4'

In [8]:
from datetime import datetime

In [9]:
def parse_gpt_api_response(article_id, system_prompt, user_prompt, api_response):
    # Load the response into a dictionary if it's a string
    if isinstance(api_response, str):
        api_response = json.loads(api_response)
    
    # Extract the required information from the response
    id = api_response.get('id')
    model = api_response.get('model')
    prompt_tokens = api_response['usage'].get('prompt_tokens')
    completion_tokens = api_response['usage'].get('completion_tokens')
    json_generated = False
    try:
        content = json.loads(api_response['choices'][0]['message']['content'])
        json_generated = True
    except:
        print(f'trying to get fixed json for articleId {article_id}')
        improved_api_response = openai.ChatCompletion.create(model = 'gpt-4', messages = [{'role': 'system', 'content': 'fix the following json to be readable using json.loads(). simply out the new json. no preamble and no postamble'}, {'role': 'user', 'content': api_response['choices'][0]['message']['content']}])
        try:
            content = json.loads(improved_api_response['choices'][0]['message']['content'])
            json_generated = True
        except:
            content = api_response['choices'][0]['message']['content']
            print(f'json not properly generated for {article_id} at {"{:%b %d, %Y %H:%M}".format(datetime.now())}')
    # Build the dictionary with the required information
    extracted_data = {
        'article_id': article_id,
        'system_prompt': system_prompt,
        'user_prompt': user_prompt,
        'model': model,
        'content': json.dumps(content),
        'prompt_tokens': prompt_tokens,
        'completion_tokens': completion_tokens,
        'proper_json_generated': json_generated
    }

    return extracted_data

In [10]:
from tqdm.notebook import tqdm

In [11]:
import json

In [12]:
model = 'gpt-4'
encoder = tiktoken.encoding_for_model(model)
num_system_tokens = len(encoder.encode(system_prompt))
output_tokens = 1024
total_tokens = 4096
available_tokens = total_tokens - num_system_tokens - output_tokens

In [43]:
def fetch_save_attributes(article_row):
    truncated_content = truncate_text_for_token_limit(text=article_row.full_content, encoder=encoder, token_limit=available_tokens)
    try:
        res_gpt = openai.ChatCompletion.create(model = model, messages = [{'role': 'system', 'content': system_prompt.replace('\n',' ')}, {'role': 'user', 'content': f'{truncated_content}|article_end|'}])
        parsed_res = parse_gpt_api_response(article_id=article_row.article_id, system_prompt=system_prompt, user_prompt=truncated_content, api_response=res_gpt)
        cur_df = pd.DataFrame(parsed_res, index = ['i',])
        cur_df.to_csv('gpt-4-responses-1-dec.csv',header=None, mode='a')
        return True
    except:
        return False

In [17]:
import time

In [38]:
def get_gpt4_attributes_for_article_ids(fraction):
    start_time = time.time()
    responses = {}
    valid_attributes = {}
    valid_responses = 0
    shortlisted_rows = shortlisted_articles.sample(frac=fraction)
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_save_attributes, article_row) for article_row in shortlisted_rows.itertuples()]
        
        for future in as_completed(futures):
            fetched = future.result()
            if fetched:  # Timeout or other errors
                valid_responses += 1
                continue
    print(f'done in {time.time() - start_time} seconds')
    return valid_responses

In [42]:
n = get_gpt4_attributes_for_article_ids(1)

trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 651e0533a662d76276b85936
trying to get fixed json for articleId 6

#### Correcting responses

In [47]:
actual_articles_df = shortlisted_articles[['article_id', 'full_content']]

In [50]:
gpt_responses = pd.read_csv('gpt-4-responses-1-dec.csv', header=None)

In [54]:
gpt_responses = gpt_responses.rename(columns = {2: 'prompt', 3: 'full_content', 4: 'model', 5: 'attributes', 6: 'prompt_tokens', 7: 'completion_tokens', 8: 'json_generated'})

In [57]:
gpt_responses['completion_tokens'].describe()

count    1581.000000
mean      507.270715
std        48.156720
min       327.000000
25%       475.000000
50%       505.000000
75%       538.000000
max       676.000000
Name: completion_tokens, dtype: float64

In [59]:
len(gpt_responses)

1581

In [61]:
gpt_responses.full_content.nunique()

1580

In [73]:
merged_gpt_responses = pd.merge(gpt_responses, actual_articles_df, how = 'inner', on = 'full_content')

In [76]:
len(merged_gpt_responses)

1517

In [75]:
merged_gpt_responses.to_csv('gpt-4-responses-1-dec-with-headers.csv', index = False)

In [41]:
len(shortlisted_articles)

1583

#### Analysis

In [24]:
all_article_responses = pd.read_csv('gpt-4-responses-22-nov.csv', header=None)

In [ ]:
all_article_responses.rename(columns = {'0'})

In [27]:
all_article_responses[1].nunique()

825

In [30]:
all_article_responses[6].sum() * 0.03 * 80/1000

4017.768

In [31]:
all_article_responses[7].sum() * 0.06 * 80/1000

2145.3984

In [32]:
invalid_response_article_ids = []

In [33]:
resp = []
for i in tqdm(range(len(all_article_responses))):
    row = all_article_responses.iloc[i]
    try:
        resp.append(json.loads(row[5]))
    except:
        invalid_response_article_ids.append(row[1])

  0%|          | 0/828 [00:00<?, ?it/s]

In [34]:
len(invalid_response_article_ids)

121

In [56]:
resp_df = pd.DataFrame(resp[:220])

In [61]:
resp_df['is_financial_or_business_news'] = resp_df['is_financial_or_business_news'].astype('int')
resp_df['relevant_for_india'] = resp_df['relevant_for_india'].astype('int')
resp_df['article_validity_duration'] = resp_df['article_validity_duration'].astype('int')

In [63]:
resp_df.is_financial_or_business_news.mean()

0.8863636363636364

In [64]:
resp_df.relevant_for_india.mean()

0.8090909090909091

In [69]:
resp_df.groupby('popularity')['popularity'].count()

popularity
breaking_news          33
moderately_popular    115
niche                  72
Name: popularity, dtype: int64

### Rerunning for invalid article responses

In [ ]:
invalid_response_article_ids

In [ ]:
shortlisted_articles

In [80]:
encoder = tiktoken.encoding_for_model(model)
num_system_tokens = len(encoder.encode(system_prompt))
total_tokens = 7000
available_tokens = total_tokens - num_system_tokens
total_rows = len(shortlisted_articles)
for i in tqdm(invalid_response_article_ids[92:]):
    row = shortlisted_articles.loc[shortlisted_articles.article_id == i].iloc[0]
    truncated_content = find_token_limit(text=row.full_content, encoder=encoder, token_limit=available_tokens)
    res_gpt = openai.ChatCompletion.create(model = model, messages = [{'role': 'system', 'content': system_prompt.replace('\n',' ')}, {'role': 'user', 'content': truncated_content}])
    parsed_res = parse_gpt_api_response(article_id=row.article_id, system_prompt=system_prompt, user_prompt=truncated_content, api_response=res_gpt)
    cur_df = pd.DataFrame(parsed_res, index = ['i',])
    cur_df.to_csv('gpt-4-responses-22-nov.csv',header=None, mode='a')
    shorted_article_assessments.append(parsed_res)

  0%|          | 0/29 [00:00<?, ?it/s]

In [500]:
df1 = pd.DataFrame(shorted_article_assessments)

In [503]:
df1['json_safe_content'] = df1['content'].apply(lambda x: json.dumps(x))

In [506]:
df1.to_csv('gpt-responses.csv')

In [510]:
df1['total_cost'] = df1['prompt_tokens'] * (0.03 * 80/1000) + df1['completion_tokens'] * (0.06 * 80/1000)

In [512]:
df1.total_cost.sum()

879.1272

In [495]:
pd.DataFrame(shorted_article_assessments).to_csv('gpt-responses.csv')

#### Standardizing responses

In [2]:
import pandas as pd
import numpy as np
import json

In [5]:
df = pd.read_csv('gpt-4-responses-22-nov.csv', header=None)

In [7]:
df = df.rename(columns = {1: 'article_id', 2: 'system_prompt', 3: 'article_content', 4: 'gpt-model', 5: 'gpt-response', 6: 'input_tokens', 7: 'output_tokens'})

In [157]:
article_wise_responses = {}

In [158]:
for i in range(len(df)):
    row = df.iloc[i]
    art_id = row['article_id']
    try:
        article_wise_responses[art_id] = {}
        article_wise_responses[art_id]['response'] = json.loads(row['gpt-response'])
        article_wise_responses[art_id]['content'] = row['article_content']
    except:
        pass

In [61]:
def correct_json_using_gpt(summary_json):
    print(f'trying to get fixed json for summary')
    improved_summary_json = openai.ChatCompletion.create(model = 'gpt-4', messages = [{'role': 'system', 'content': 'fix the following json to be readable using json.loads(). simply out the new json. no preamble and no postamble'}, {'role': 'user', 'content': summary_json}])
    content = json.loads(improved_summary_json['choices'][0]['message']['content'])
    return content

In [156]:
from tqdm.notebook import tqdm
import tqdm as td

from copy import deepcopy

In [185]:
response['summary_list']

[{'missing_entities': 'Nirmala Sitharaman; US India Business Council; Digitalization impact',
  'denser_summary': 'Union Finance Minister Nirmala Sitharaman urged American businesses to invest in India, asserting that it meets the requirements of a fair and transparent economy. The invitation came during a business roundtable organized by the US India Business Council. Sitharaman emphasized on the progress India has made in terms of digitalization and economic formalization.'},
 {'missing_entities': 'Taranjit Singh Sandhu; US and India trade; National Financial Information Registry',
  'denser_summary': "Indian Ambassador to the US, Taranjit Singh Sandhu reiterated India's commitment to engagement at the G20. He also noted that the US-India bilateral trade crossed $190 billion last year. The importance of a national financial information registry was emphasized to expand credit access."},
 {'missing_entities': 'Startup policies; Public Infrastructure; Fintech Ecosystem',
  'denser_summ

In [190]:
standardized_responses = {}
errenous_summaries = []
for art_id in td.tqdm(article_wise_responses.keys()):
    response = article_wise_responses[art_id]['response']
    expected_keys_1 = ['analysis_is_financial_or_business_news', 'is_financial_or_business_news', 'analysis_of_relevant_for_india', 'relevant_for_india',
                        'analysis_of_article_validity_duration', 'article_validity_duration', 'analysis_of_popularity', 'popularity', 'analysis_of_article_type', 'article_type', 
                        'analysis_of_article_sentiment', 'article_sentiment', 'headline_suggestion']
    expected_keys_2 = ['summary_list']
    if isinstance(response, list):
        res = {}
        for ele in response:
            res.update(ele)
        response = res
        # response = deepcopy(article_wise_responses[art_id]['response'][0])
        # try:
        #     response.update(article_wise_responses[art_id]['response'][1])
        # except:
        #     pass
        # assert sorted(set(response[0].keys()).intersection(expected_keys_1)) == sorted(expected_keys_1), f"{art_id}"
        # assert set(response[1].keys()) == expected_keys_2
        # try:
        #     summaries = json.loads(response[1]['summary_list'])
        # except:
        #     errenous_summaries.append(art_id)
        #     continue
        #     # print(f'articleId is {art_id}')
        #     # summaries = correct_json_using_gpt(summary_json = response[1]['summary_list'])
        # assert isinstance(summaries, list)
        # assert len(summaries) == 3
        # for summary in summaries:
        #     assert 'missing_entities' in summary and 'denser_summary' in summary
        # standardized_responses[art_id] = []
        # standardized_responses[art_id].append(response[1])
        # standardized_responses[art_id].append(summaries)
        # elif isinstance(response, dict):
    try:
        assert sorted(set(response.keys()).intersection(set(expected_keys_1).union(set(expected_keys_2)))) == sorted(set(expected_keys_1).union(set(expected_keys_2)))
    except:
        errenous_summaries.append(art_id)
        print(f'{art_id} error 0')
        continue
        # print(f'articleId is {art_id}')
        # summaries = correct_json_using_gpt(summary_json = response['summary_list'])
    try:
        if isinstance(response['summary_list'], str):
            summaries = json.loads(response['summary_list'])
        elif isinstance(response['summary_list'], list):
            summaries = response['summary_list']
    except:
        corrected_summary = correct_json_using_gpt(response['summary_list'])
        try:
            summaries = corrected_summary
        except:
            errenous_summaries.append(art_id)
            print(f'{art_id} error 1')
            continue
    assert isinstance(summaries, list)
    try:
        assert len(summaries) == 3
    except:
        print(f'{art_id} error 2')
        errenous_summaries.append(art_id)
        continue
    for summary in summaries:
        try:
            assert 'missing_entities' in summary and ('denser_summary' in summary or 'summary_text' in summary)
            if 'summary_text' in summary:
                summary['denser_summary'] = summary.pop('summary_text')
        except:
            print(f'{art_id} error 3')
            errenous_summaries.append(art_id)
            continue
    standardized_responses[art_id] = {}
    standardized_responses[art_id]['response'] = []
    attribute_response = {}
    for key in expected_keys_1:
        attribute_response[key] = response[key]
    standardized_responses[art_id]['response'].append(attribute_response)
    standardized_responses[art_id]['response'].append(summaries)
    standardized_responses[art_id]['content'] = article_wise_responses[art_id]['content']

  0%|          | 0/825 [00:00<?, ?it/s]

trying to get fixed json for summary


  0%|          | 3/825 [00:15<1:12:02,  5.26s/it]

trying to get fixed json for summary


  0%|          | 4/825 [00:35<2:16:53, 10.00s/it]

...

100%|█████████▉| 821/825 [1:07:31<00:27,  6.81s/it]

trying to get fixed json for summary


100%|█████████▉| 822/825 [1:07:55<00:27,  9.17s/it]

trying to get fixed json for summary


100%|██████████| 825/825 [1:08:09<00:00,  4.96s/it]


In [163]:
list(standardized_responses.keys())[-20:]

['6555ccc44b13023f934acd61',
 '651e0c73a662d76276b86a15',
 '651dd71da662d76276b7e3c2',
 '6555cb9f4b13023f9349fa71',
 '6555cba44b13023f9349fdd1',
 '6555cab84b13023f93495f7c',
 '6555c7ec4b13023f9348ee4a',
 '654cbbcadc4fa72a6c403e10',
 '6555c0084b13023f9348c97b',
 '6555ccd14b13023f934ad8ab',
 '651df9b1a662d76276b83c5d',
 '6555cbf64b13023f934a3718',
 '65316f1c1e5cc42b1b13efc1',
 '651e088ba662d76276b86119',
 '653168bd1e5cc42b1b13b78d',
 '655c5595eee55a44e0ac1e51',
 '6555c84a4b13023f934924ce',
 '651dca7ca662d76276b7c1c9',
 '651df878a662d76276b83979',
 '6555c80e4b13023f934901a6']

In [191]:
len(standardized_responses.keys())

821

In [192]:
len(errenous_summaries)

4

In [195]:
test_article_ids = list(df.iloc[:101].article_id.values)

In [196]:
len(set(test_article_ids).intersection(set(standardized_responses.keys())))

100

In [197]:
train_article_ids = [x for x in standardized_responses.keys() if x not in test_article_ids]

In [201]:
len(train_article_ids)

721

In [202]:
train_standardized_responses = {k: standardized_responses[k] for k in train_article_ids}

In [203]:
len(train_standardized_responses)

721

In [204]:
with open('train_set_full_27_nov.json', 'w') as f:
    json.dump(train_standardized_responses, f)

In [205]:
test_standardized_responses = {k: standardized_responses[k] for k in test_article_ids}

In [206]:
with open('test_set_full_27_nov.json', 'w') as f:
    json.dump(test_standardized_responses, f)